# Diplomatura en Ciencia de Datos, Aprendizaje Automático y sus Aplicaciones

## Proyecto de Mentoría: _Predicción de Series Temporales Financieras con Machine Learning_

### Trabajo Práctico 2: _Exploración y curación de datos_

Grupo 1:

- Juan Cruz Gonzalez
- Marcelo Fernando Fullana Jornet
- Emanuel Nicolás Herrador
- Griselda Itovich
- Ariel Maximiliano Pereira

---


## Índice


- [1. Introducción](#1-introducción)
- [2. Desarrollo previo](#2-desarrollo-previo)
  - [2.1. Importación de librerías](#21-importación-de-librerías)
  - [2.2. Carga de datos](#22-carga-de-datos)
  - [2.3. Funciones auxiliares](#23-funciones-auxiliares)
- [3. Análisis exploratorio inicial](#3-análisis-exploratorio-inicial)
  - [3.1. Análisis de los precios de cierre ajustados](#31-análisis-de-los-precios-de-cierre-ajustados)
  - [3.2. Análisis de los retornos diarios](#32-análisis-de-los-retornos-diarios)
- [4. Normalización de series temporales](#4-normalización-de-series-temporales)
  - [4.1. Min-Max Scaling](#41-min-max-scaling)
  - [4.2. Z-Score Normalization](#42-z-score-normalization)
- [5. Valores atípicos](#5-valores-atípicos)
  - [5.1. Método del rango intercuartílico](#51-método-del-rango-intercuartílico)
  - [5.2. Método de Z-Score](#52-método-de-z-score)
  - [5.3. Comparación de métodos](#53-comparación-de-métodos)
- [6. Manejo de valores faltantes](#6-manejo-de-valores-faltantes)
- [7. Análisis de correlación](#7-análisis-de-correlación)
- [8. Análisis de AutoCorrelación y AutoCorrelación Parcial](#8-análisis-de-autocorrelación-y-autocorrelación-parcial)
- [9. Suavizamiento de Series Temporales](#9-suavizamiento-de-series-temporales)
  - [9.1. Suavizamiento exponencial](#91-suavizamiento-exponencial)
  - [9.2. Suavizamiento mediante medias móviles](#92-suavizamiento-mediante-medias-móviles)
  - [9.3. Comparación de métodos de suavizamiento](#93-comparación-de-métodos-de-suavizamiento)
- [10. Análisis de estacionalidad](#10-análisis-de-estacionalidad)


## 1. Introducción


En el presente trabajo, se realizará la exploración y curación de los datos de la serie temporal de datos de precios de las acciones de la empresa Tesla Inc. (TSLA), que cotiza en la bolsa de valores NASDAQ desde el 29 de junio de 2010. Los datos que se tendrán en cuenta serán los [mismos](https://raw.githubusercontent.com/Emmatassone/Mentoria_FaMAF_2024/main/tsla_split_adjusted.csv) que en el trabajo práctico anterior (Análisis y Visualización), por lo que corresponden al período comprendido entre el 29 de junio de 2010 y el 15 de abril de 2024.

El objetivo de este trabajo es realizar un análisis exploratorio de los datos, identificar problemas de calidad y completitud, y aplicar técnicas de curación de datos para prepararlos para la etapa de modelado. Por ello mismo, nos enfocaremos en la identificación y tratamiento de valores faltantes, valores atípicos, normalización de los datos, análisis de correlación, suavizado de la serie temporal y análisis de estacionalidad.

_Nota_: A lo largo del proyecto se observará que el período más relevante para el análisis de las variables de _close_ y _change_percent_ es el comprendido entre el 1 de enero de 2020 y el 15 de abril de 2024. Por ello, será en este período en el que se centrará el análisis exploratorio y la curación de datos.


## 2. Desarrollo previo


### 2.1. Importación de librerías


Las librerías que usaremos en el presente proyecto son las siguientes:


In [1]:
# Importamos las librerías
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Escalamiento de los datos
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# Valores faltantes
import missingno as msno

# AutoCorrelación
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# Seteamos la seed para reproducibilidad
np.random.seed(0)

### 2.2. Carga de datos


Ahora, se procederá a cargar el dataset _tsla_split_adjusted_ para tenerlo disponible en la realización del trabajo práctico.


In [2]:
file_url = 'https://raw.githubusercontent.com/Emmatassone/Mentoria_FaMAF_2024/main/tsla_split_adjusted.csv'
df = pd.read_csv(file_url)

# Colocar la fecha como tipo datetime
df['date'] = pd.to_datetime(df['date'])

# Colocar los demás valores de tipo float excepto volume
float_features = ['open', 'high', 'low', 'close',
                  'raw_close', 'change_percent', 'avg_vol_20d']
int_features = ['volume']

for feature in float_features:
    df[feature] = df[feature].astype(float)
for feature in int_features:
    df[feature] = df[feature].astype(int)

# Mostrar las primeras filas del dataset para chequear
df.head()

,date,open,high,low,close,volume,raw_close,change_percent,avg_vol_20d
0,2010-06-29,1.26666,1.66666,1.16934,1.59266,281749173,23.8899,NaN,NaN
1,2010-06-30,1.71934,2.02794,1.55334,1.58866,257915884,23.8299,-0.25,NaN
2,2010-07-01,1.66666,1.72800,1.35134,1.46400,123447940,21.9600,-7.85,NaN
3,2010-07-02,1.53334,1.54000,1.24734,1.28000,77127102,19.2000,-12.57,NaN
4,2010-07-06,1.33334,1.33334,1.05534,1.07400,103189435,16.1100,-16.09,NaN


### 2.3. Funciones auxiliares


Para hacer más legible el código y más sencilla la realización de este laboratorio, haremos uso de las siguientes funciones auxiliares:


In [3]:
def plot_simple_scatter(df, x, y, title, x_title, y_title, rect=[], values=[], log=False, df_bg=None, df_bg_title=None):
    """
    Función para graficar un scatter plot con Plotly.

    Parámetros:
        [+] df: DataFrame con los datos.
        [+] x: Nombre de la columna a graficar en el eje x.
        [+] y: Nombre de la columna a graficar en el eje y.
        [+] title: Título del gráfico.
        [+] x_title: Título del eje x.
        [+] y_title: Título del eje y.
        [+] rect: Lista de listas con los rectángulos a dibujar (x0, x1, color).
        [+] values: Lista de listas con los valores a dibujar (y, estilo, color, texto, posición).
        [+] log: Booleano para indicar si se quiere escala logarítmica en el eje y.
        [+] df_bg: DataFrame con los datos de fondo.
        [+] df_bg_title: Título del gráfico de fondo.

    Retorno:
        [+] fig: Objeto de tipo Figure con el gráfico.
    """
    # En caso de querer escala logarítmica
    if log:
        df2 = df.copy()
        df2[y] = np.log(df[y])

        df_bg2 = None
        if df_bg is not None:
            df_bg2 = df_bg.copy()
            df_bg2[y] = np.log(df_bg[y])

        return plot_simple_scatter(df2, x, y, title, x_title, y_title, rect, values, log=False, df_bg=df_bg2, df_bg_title=df_bg_title)

    # Creamos la figura
    fig = go.Figure()

    # Obtenemos los valores máximos y mínimos
    max_y = df[y].max()
    min_y = df[y].min()

    # Agregamos los gráficos
    if df_bg is not None:
        fig.add_trace(go.Scatter(
            x=df_bg[x], y=df_bg[y], mode='lines', name=df_bg_title, line=dict(color='coral')))

    fig.add_trace(go.Scatter(x=df[x], y=df[y], mode='lines',
                  name=y_title, line=dict(color='royalblue')))

    # Actualizamos el layout
    fig.update_layout(title=title, xaxis_title=x_title, yaxis_title=y_title)

    # Agregamos los rectángulos
    for x in rect:
        fig.add_shape(type='rect', x0=x[0], y0=min_y, x1=x[1],
                      y1=max_y, fillcolor=x[2], opacity=0.5, layer='below')

    # Agregamos los valores
    for v in values:
        if v[0].isnumeric():
            val_y = df[y].quantile(float(v[0]) / 100)
        else:
            val_y = eval(f'np.{v[0]}')(df[y])

        text = f'{v[3]}: {val_y:.2f}'
        fig.add_hline(y=val_y, line_dash=v[1], line_color=v[2], annotation_text=text,
                      annotation_position=v[4], annotation_font=dict(color=v[2]), layer='below')

    return fig

In [4]:
def plot_simple_boxplot(df, x, title, x_label, show_points=False, with_mean_std=False):
    """
    Función para graficar un boxplot con Plotly.

    Parámetros:
        [+] df: DataFrame con los datos.
        [+] x: Nombre de la columna a graficar en el eje x.
        [+] title: Título del gráfico.
        [+] x_label: Título del eje x.
        [+] show_points: Booleano para indicar si se muestran los puntos.

    Retorno:
        [+] fig: Objeto de tipo Figure con el gráfico.
    """
    # Creamos la figura
    fig = go.Figure()

    # Agregamos el gráfico
    boxpoints = 'all' if show_points else 'outliers'
    boxmean = 'sd' if with_mean_std else False
    fig.add_trace(
        go.Box(x=df[x], name=x, boxpoints=boxpoints, boxmean=boxmean))

    # Actualizamos el layout
    fig.update_layout(title=title, xaxis_title=x_label)

    return fig

In [5]:
def plot_subplots(fig_list, title, height=600, showlegend=False):
    """
    Función para graficar varios gráficos en un mismo layout, uno debajo del otro.

    Parámetros:
        [+] fig_list: Lista de objetos de tipo Figure.
        [+] title: Título del gráfico.
        [+] height: Alto del gráfico.

    Retorno:
        [+] fig: Objeto de tipo Figure con el gráfico.
    """
    fig = make_subplots(rows=len(fig_list), cols=1, subplot_titles=[
                        fig['layout']['title']['text'] for fig in fig_list], shared_xaxes=True, vertical_spacing=0.1)

    for i, f in enumerate(fig_list):
        for trace in f.data:
            fig.add_trace(trace, row=i + 1, col=1)

    fig.update_layout(title=title, height=height)
    fig.update_layout(showlegend=showlegend)

    return fig

In [6]:
# Los vamos a hacer de forma interactiva con plotly, tal y como se describe en este foro:
#           https://community.plotly.com/t/plot-pacf-plot-acf-autocorrelation-plot-and-lag-plot/24108/4)
#
# Esto nos permite mayor comodidad a la hora de analizar los gráficos.

def plot_acf_pacf(df, x, nlags=None):
    """
    Función para graficar la Autocorrelación y la Autocorrelación Parcial.

    Parámetros:
        [+] df: DataFrame con los datos.
        [+] x: Nombre de la columna a graficar.
        [+] nlags: Cantidad de lags a considerar.

    Retorno:
        [+] fig: Objeto de tipo Figure con los gráficos.
    """
    def create_base_plot(corr_array):
        lower_y = corr_array[1][:, 0] - corr_array[0]
        upper_y = corr_array[1][:, 1] - corr_array[0]

        fig = go.Figure()
        [fig.add_scatter(x=(x, x), y=(0, corr_array[0][x]), mode='lines', line_color='#3f3f3f')
         for x in range(len(corr_array[0]))]
        fig.add_scatter(x=np.arange(len(corr_array[0])), y=corr_array[0], mode='markers', marker_color='#1f77b4',
                        marker_size=12)
        fig.add_scatter(x=np.arange(
            len(corr_array[0])), y=upper_y, mode='lines', line_color='rgba(255,255,255,0)')
        fig.add_scatter(x=np.arange(len(corr_array[0])), y=lower_y, mode='lines', fillcolor='rgba(32, 146, 230,0.3)',
                        fill='tonexty', line_color='rgba(255,255,255,0)')
        fig.update_traces(showlegend=False)
        fig.update_xaxes(range=[-1, nlags + 1])
        fig.update_yaxes(zerolinecolor='#000000')

        return fig

    # ACF
    corr_acf = sm.tsa.acf(df[x], nlags=nlags, alpha=0.05)
    fig_acf = create_base_plot(corr_acf)
    fig_acf.update_layout(title=f'Autocorrelación de {x}')

    # PACF
    corr_pacf = sm.tsa.pacf(df[x], nlags=nlags, alpha=0.05)
    fig_pacf = create_base_plot(corr_pacf)
    fig_pacf.update_layout(title=f'Autocorrelación Parcial de {x}')

    return fig_acf, fig_pacf

## 3. Análisis exploratorio inicial


Ya habiendo realizado una primera aproximación a los datos en el trabajo práctico anterior, ahora nos concentraremos en obtener conclusiones más profundas sobre la serie temporal de precios de las acciones de Tesla Inc. (TSLA). Para ello, comenzaremos con un análisis exploratorio inicial de los datos.

En este sentido, nos concentraremos en analizar los precios de cierre ajustados de las acciones y los retornos diarios correspondientes, para poder identificar cuándo una acción de tesla podría considerarse "barata" o "cara" y cuál es la caída de precios diaria que un inversor puede estar dispuesto a tolerar al invertir en esta acción.


### 3.1. Análisis de los precios de cierre ajustados


Primero tenemos que encontrar qué período vamos a considerar para este análisis, dado que no se puede decir que una acción esté cara o barata a día de hoy con datos de hace $10$ años. Por ello mismo, vamos a graficar todo el período de tiempo disponible y encontrar el óptimo para realizar este análisis.


In [7]:
# Precio de cierre para todo el período
rect = [
    ['2020-01-01', '2024-04-15', 'LightSkyBlue'],
]
values = [
    ['max', 'dot', 'blue', 'Máximo', 'top right'],
    ['min', 'dot', 'red', 'Mínimo', 'bottom right'],
]

fig = plot_simple_scatter(df, 'date', 'close', 'Precios de cierres diarios de Tesla',
                          'Fecha', 'Precio de cierre', log=False, rect=rect, values=values)
fig.show()

Gracias a ello, entonces, se puede notar que es en el año $2020$ donde se da el movimiento alcista más fuerte de la acción de Tesla, llegando a su máximo histórico en _Noviembre del_ $2021$. Por este motivo, entonces, consideraremos el período comprendido entre el $1$ de _Enero del_ $2020$ y el $15$ de _Abril del_ $2024$ para realizar el análisis de precios de cierre ajustados y retornos diarios.


In [8]:
df_2020_2024 = df.copy()
df_2020_2024 = df_2020_2024[(df_2020_2024['date'] >= '2020-01-01')]

# Resumen estadístico de los precios de cierre
display(pd.DataFrame(df_2020_2024['close'].describe()).T)

# Scatter plot de los precios de cierre
rect = []
values = [
    ['max', 'dot', 'blue', 'Máximo', 'top right'],
    ['min', 'dot', 'red', 'Mínimo', 'bottom right'],
    ['mean', 'dash', 'black', 'Promedio', 'bottom left'],
    ['25', 'dash', 'green', 'Q1', 'bottom left'],
    ['50', 'dash', 'purple', 'Q2', 'top left'],
    ['75', 'dash', 'orange', 'Q3', 'top left'],
]

fig = plot_simple_scatter(df_2020_2024, 'date', 'close', 'Precios de cierres diarios de Tesla (2020-2024)',
                          'Fecha', 'Precio de cierre', log=False, rect=rect, values=values)
fig.show()

,count,mean,std,min,25%,50%,75%,max
close,1078.0,207.953595,83.180977,24.08134,166.1675,219.475,259.9325,409.97


Vistos los precios de cierres de este período, entonces podemos notar cómo han variado en los últimos años y su distribución a lo largo del tiempo. Además, otra cosa a considerar es que el período está bien seleccionado dado que la mediana y la media de los precios tienen valores cercanos ($219.47$ vs. $207.95$), por lo que no hay una gran dispersión en los datos.

Por ello mismo, entonces, consideraremos que un precio "normal" para las acciones de Tesla es aquél que se encuentra dentro del rango intercuartílico, es decir, entre el $Q1$ y el $Q3$ de los precios, y que un precio "barato" es aquel que se encuentra por debajo del $Q1$ y un precio "caro" es aquel que se encuentra por encima del $Q3$.

De ese modo, entonces, tenemos que un precio $X$ de una acción de Tesla se considerará:

- _Normal_: si $166.17\leq X\leq 259.93$
- _Barato_: si $X<166.17$
- _Caro_: si $X>259.93$

Podemos verlo, también, con un _box plot_:


In [9]:
# Boxplot de los precios de cierre
fig = plot_simple_boxplot(df_2020_2024, 'close', 'Distribución de precios de cierre de Tesla (2020-2024)',
                          'Precio de cierre', show_points=True, with_mean_std=True)
fig.show()

Donde podemos ver la cantidad de momentos donde la acción de Tesla fue considerada "barata" o "cara" en el período seleccionado.


### 3.2. Análisis de los retornos diarios


Ahora, nos concentraremos en analizar cuál es la caída de precios diaria que un inversor tendría que estar dispuesto a tolerar al invertir en esta acción. Para ello, consideraremos el mismo período $2020$ - $2024$ tomado en el anterior punto y veremos cómo se distribuyen en general y a lo largo del tiempo.


In [10]:
# Scatter plot de los retornos diarios
rect = []
values = [
    ['max', 'dot', 'blue', 'Máximo', 'top right'],
    ['min', 'dot', 'red', 'Mínimo', 'bottom right'],
]

fig = plot_simple_scatter(df_2020_2024, 'date', 'change_percent', 'Retornos diarios de Tesla (2020-2024)',
                          'Fecha', 'Retorno diario', log=False, rect=rect, values=values)
fig.show()

Gracias a lo cual podemos notar la gran variabilidad que hubo en los retornos diarios en los años $2020$ y $2021$, y cómo luego se encuentran más estables en los años $2022$ y $2023$.

Ahora, respecto a la distribución de los retornos, podemos visualizarlo con un _box plot_:


In [11]:
# Resumen estadístico de los retornos diarios
display(pd.DataFrame(df_2020_2024['change_percent'].describe()).T)

# Boxplot de los retornos diarios
fig = plot_simple_boxplot(df_2020_2024, 'change_percent', 'Distribución de retornos diarios de Tesla (2020-2024)',
                          'Retorno diario', show_points=True, with_mean_std=True)
fig.show()

,count,mean,std,min,25%,50%,75%,max
change_percent,1078.0,0.251911,4.217742,-21.06,-2.02,0.195,2.2575,19.89


De donde notamos que podríamos tomar que un inversor tolerará una caída de precios diaria mayor o igual a $Q1$, es decir, menor al $2.02\%$.


## 4. Normalización de series temporales


Algo que podemos hacer para mejorar la calidad de nuestros datos a la hora de usarlos en modelos de machine learning es escalarlos o normalizarlos. Esto se realiza con el objetivo de que la escala de los datos no influya en el modelo y que la importancia de cada uno se dé por su relación con el _target_ que se quiere predecir.

Por ello mismo, vamos a probar dos normalizaciones distintas:

- Min-Max Scaling
- Z-Score Normalization

_Observación_: notar que el período seleccionado para realizar este análisis es el mismo que el seleccionado en el punto anterior, es decir, el comprendido entre el $1$ de _Enero del_ $2020$ y el $15$ de _Abril del_ $2024$.


### 4.1. Min-Max Scaling


Si aplicamos el Min-Max Scaling a nuestros datos, entonces estos se encontrarán en el rango $[0, 1]$. Veamos cómo se ven los precios de cierre ajustados y los retornos diarios escalados:


In [12]:
# Escalamos los datos
df_2020_2024['close_scaled_minmax'] = MinMaxScaler().fit_transform(
    df_2020_2024[['close']])
df_2020_2024['change_percent_scaled_minmax'] = MinMaxScaler().fit_transform(
    df_2020_2024[['change_percent']])

# Scatter plot de precios de cierre y retornos diarios escalados
rect = []
values1 = [
    ['max', 'dot', 'blue', 'Máximo', 'top right'],
    ['min', 'dot', 'red', 'Mínimo', 'bottom right'],
    ['mean', 'dash', 'black', 'Promedio', 'bottom left'],
    ['25', 'dash', 'green', 'Q1', 'bottom left'],
    ['50', 'dash', 'purple', 'Q2', 'top left'],
    ['75', 'dash', 'orange', 'Q3', 'top left'],
]
values2 = [
    ['max', 'dot', 'blue', 'Máximo', 'top right'],
    ['min', 'dot', 'red', 'Mínimo', 'bottom right'],
]

fig1 = plot_simple_scatter(df_2020_2024, 'date', 'close_scaled_minmax', 'Precios de cierres diarios de Tesla escalados (2020-2024)',
                           'Fecha', 'Precio de cierre escalado', log=False, rect=rect, values=values1)
fig2 = plot_simple_scatter(df_2020_2024, 'date', 'change_percent_scaled_minmax', 'Retornos diarios de Tesla escalados (2020-2024)',
                           'Fecha', 'Retorno diario escalado', log=False, rect=rect, values=values2)

fig1.show()
fig2.show()

Lo cual muestra la misma relación entre los datos, pero ahora en un rango más acotado. Por ejemplo, la distribución debe ser análoga a la original, tal y como se muestra con el siguiente box plot:


In [13]:
# Resumen estadístico de los precios de cierre y los retornos diarios escalados
display(pd.concat([pd.DataFrame(df_2020_2024['close_scaled_minmax'].describe(
)).T, pd.DataFrame(df_2020_2024['change_percent_scaled_minmax'].describe()).T]))

# Boxplot de los precios de cierre y los retornos diarios escalados
fig1 = plot_simple_boxplot(df_2020_2024, 'close_scaled_minmax', 'Distribución de precios de cierre escalados de Tesla (2020-2024)',
                           'Precio de cierre escalado', show_points=True, with_mean_std=True)
fig2 = plot_simple_boxplot(df_2020_2024, 'change_percent_scaled_minmax', 'Distribución de retornos diarios escalados de Tesla (2020-2024)',
                           'Retorno diario escalado', show_points=True, with_mean_std=True)

fig1.show()
fig2.show()

,count,mean,std,min,25%,50%,75%,max
close_scaled_minmax,1078.0,0.476490,0.215557,0.0,0.368205,0.506347,0.611190,1.0
change_percent_scaled_minmax,1078.0,0.520437,0.102997,0.0,0.464957,0.519048,0.569414,1.0


### 4.2. Z-Score Normalization


Ahora, si aplicamos la normalización Z-Score a nuestros datos, entonces estos tendrán una media de $0$ y una desviación estándar de $1$. Veamos cómo se ven los precios de cierre ajustados y los retornos diarios normalizados:


In [14]:
# Normalizamos los datos
df_2020_2024['close_scaled_zscore'] = StandardScaler().fit_transform(
    df_2020_2024[['close']])
df_2020_2024['change_percent_scaled_zscore'] = StandardScaler().fit_transform(
    df_2020_2024[['change_percent']])

# Scatter plot de precios de cierre y retornos diarios normalizados
rect = []
values1 = [
    ['max', 'dot', 'blue', 'Máximo', 'top right'],
    ['min', 'dot', 'red', 'Mínimo', 'bottom right'],
    ['mean', 'dash', 'black', 'Promedio', 'bottom left'],
    ['25', 'dash', 'green', 'Q1', 'bottom left'],
    ['50', 'dash', 'purple', 'Q2', 'top left'],
    ['75', 'dash', 'orange', 'Q3', 'top left'],
]
values2 = [
    ['max', 'dot', 'blue', 'Máximo', 'top right'],
    ['min', 'dot', 'red', 'Mínimo', 'bottom right'],
]

fig1 = plot_simple_scatter(df_2020_2024, 'date', 'close_scaled_zscore', 'Precios de cierres diarios de Tesla normalizados (2020-2024)',
                           'Fecha', 'Precio de cierre normalizado', log=False, rect=rect, values=values1)
fig2 = plot_simple_scatter(df_2020_2024, 'date', 'change_percent_scaled_zscore', 'Retornos diarios de Tesla normalizados (2020-2024)',
                           'Fecha', 'Retorno diario normalizado', log=False, rect=rect, values=values2)
fig1.show()
fig2.show()

Y es gracias a esta normalización que incluso puede notarse la existencia de enormes valores atípicos en los precios de cierre y en los retornos diarios, dado que se encuentran a una distancia igual a $2.43$ y $5.06$ desviaciones estándar de la media, respectivamente.

Esto se puede ver, también, en el siguiente box plot (el cual resulta análogo a los anteriores, obviamente):


In [15]:
# Resumen estadístico de los precios de cierre y los retornos diarios normalizados
display(pd.concat([pd.DataFrame(df_2020_2024['close_scaled_zscore'].describe(
)).T, pd.DataFrame(df_2020_2024['change_percent_scaled_zscore'].describe()).T]))

# Boxplot de los precios de cierre y los retornos diarios normalizados
fig1 = plot_simple_boxplot(df_2020_2024, 'close_scaled_zscore', 'Distribución de precios de cierre normalizados de Tesla (2020-2024)',
                           'Precio de cierre normalizado', show_points=True, with_mean_std=True)
fig2 = plot_simple_boxplot(df_2020_2024, 'change_percent_scaled_zscore', 'Distribución de retornos diarios normalizados de Tesla (2020-2024)',
                           'Retorno diario normalizado', show_points=True, with_mean_std=True)
fig1.show()
fig2.show()

,count,mean,std,min,25%,50%,75%,max
close_scaled_zscore,1078.0,3.691131e-16,1.000464,-2.211534,-0.502585,0.138574,0.625179,2.429764
change_percent_scaled_zscore,1078.0,-6.591306e-18,1.000464,-5.055265,-0.538906,-0.013499,0.475733,4.658228


La ventaja de usar la normalización con Z-score (es decir, con `StandardScaler`) es que podemos ver cómo se distribuyen los datos en relación a la media y la desviación estándar, lo cual nos permite identificar valores atípicos de manera más sencilla.


## 5. Valores atípicos


Para el descarte de los valores atípicos en nuestros datos, vamos a considerar dos métodos distintos:

- Método del _rango intercuartílico_
- Método de _Z-Score_

para realizar el descarte de estos valores "extremos" en nuestros datos. Al igual que como vimos antes, estos pueden visualizarse con el uso de _box plots_.

_Observación_: notar que el período de tiempo seleccionado sigue siendo $2020$ - $2024$.


In [16]:
# Boxplot de los precios de cierre y los retornos diarios
fig1 = plot_simple_boxplot(df_2020_2024, 'close', 'Distribución de precios de cierre de Tesla (2020-2024)',
                           'Precio de cierre', show_points=True, with_mean_std=True)
fig2 = plot_simple_boxplot(df_2020_2024, 'change_percent', 'Distribución de retornos diarios de Tesla (2020-2024)',
                           'Retorno diario', show_points=True, with_mean_std=True)
fig1.show()
fig2.show()

Luego, entonces, vamos a ver cómo se ven los precios de cierre ajustados y los retornos diarios sin los valores atípicos para ambos métodos.


### 5.1. Método del rango intercuartílico


Como sabemos, el método del rango intercuartílico (IQR) se basa en la diferencia entre el tercer y el primer cuartil de los datos. Si un valor se encuentra por debajo de $Q1 - 1.5 \times IQR$ o por encima de $Q3 + 1.5 \times IQR$, entonces se considera un valor atípico (es lo que en el gráfico se muestra como _outliers_, fuera de los bigotes del _box plot_).

Por ello, hagamos la limpieza de los datos con este método y veamos cómo se ven los precios de cierre ajustados y los retornos diarios sin los valores atípicos.


In [17]:
df_2020_2024_iqr = df_2020_2024.copy()

# IQR Cleaning
Q1_close = df_2020_2024['close'].quantile(0.25)
Q3_close = df_2020_2024['close'].quantile(0.75)
IQR_close = Q3_close - Q1_close
IQR_close_condition = (df_2020_2024['close'] >= Q1_close - 1.5 * IQR_close) & (
    df_2020_2024['close'] <= Q3_close + 1.5 * IQR_close)

Q1_change_percent = df_2020_2024['change_percent'].quantile(0.25)
Q3_change_percent = df_2020_2024['change_percent'].quantile(0.75)
IQR_change_percent = Q3_change_percent - Q1_change_percent
IQR_change_percent_condition = (df_2020_2024['change_percent'] >= Q1_change_percent - 1.5 * IQR_change_percent) & (
    df_2020_2024['change_percent'] <= Q3_change_percent + 1.5 * IQR_change_percent)

df_2020_2024_iqr = df_2020_2024_iqr[IQR_close_condition &
                                    IQR_change_percent_condition]

# Scatter plot de los precios de cierre y los retornos diarios (ALL vs. IQR cleaning)
values = [
    ['max', 'dot', 'blue', 'Máximo', 'top right'],
    ['min', 'dot', 'red', 'Mínimo', 'bottom right'],
]
fig1 = plot_simple_scatter(df_2020_2024_iqr, 'date', 'close', 'Precios de cierre diarios de Tesla con valores atípicos vs. sin (IQR cleaning) (2020-2024)',
                           'Fecha', 'Precio de cierre', log=False, rect=[], values=values, df_bg=df_2020_2024, df_bg_title='Originales')
fig2 = plot_simple_scatter(df_2020_2024_iqr, 'date', 'change_percent', 'Retornos diarios de Tesla con valores atípicos vs. sin (IQR cleaning) (2020-2024)',
                           'Fecha', 'Retorno diario', log=False, rect=[], values=values, df_bg=df_2020_2024, df_bg_title='Originales')

fig1.show()
fig2.show()

Gracias a lo cual podemos ver cómo se ven los precios de cierre ajustados y los retornos diarios sin los valores atípicos, y cómo se distribuyen en general y a lo largo del período 2020 - 2024. Notar que la mayor limpieza se da en los retornos diarios, estando estos ahora en el rango $[-8.33, 8.65]$.

La limpieza puede verse comparando los box plots de los precios de cierre ajustados y los retornos diarios con y sin los valores atípicos:


In [18]:
# Boxplot de los precios de cierre (ALL vs. IQR cleaning)
fig_orig = plot_simple_boxplot(df_2020_2024, 'close', 'Con outliers',
                               'Precio de cierre', show_points=True, with_mean_std=True)
fig_iqr = plot_simple_boxplot(df_2020_2024_iqr, 'close', 'Sin outliers (IQR cleaning)',
                              'Precio de cierre', show_points=True, with_mean_std=True)

fig = plot_subplots([fig_orig, fig_iqr],
                    'Distribución de precios de cierre de Tesla con y sin outliers (2020-2024)', height=600)
fig.show()

# Boxplot de los retornos diarios (ALL vs. IQR cleaning)
fig_orig = plot_simple_boxplot(df_2020_2024, 'change_percent', 'Con outliers',
                               'Retorno diario', show_points=True, with_mean_std=True)
fig_iqr = plot_simple_boxplot(df_2020_2024_iqr, 'change_percent',
                              'Sin outliers (IQR cleaning)', 'Retorno diario', show_points=True, with_mean_std=True)

fig = plot_subplots([fig_orig, fig_iqr],
                    'Distribución de retornos diarios de Tesla con y sin outliers (2020-2024)', height=600)
fig.show()

Gracias a lo cual podemos apreciar aún más la limpieza de _outliers_ realizada para este período.


### 5.2. Método de Z-Score


El método de limpieza de valores atípicos con Z-Score se basa en la cantidad de desviaciones estándar que un valor se encuentra de la media. Si un valor se encuentra a más de $3$ desviaciones estándar de la media, entonces se considera un valor atípico.

Por ello, hagamos la limpieza de los datos con este método y veamos cómo se ven los precios de cierre ajustados y los retornos diarios sin los valores atípicos.


In [19]:
df_2020_2024_zscore = df_2020_2024.copy()

# Z-score Cleaning
zscore_close_condition = (df_2020_2024['close_scaled_zscore'] >= -3) & (
    df_2020_2024['close_scaled_zscore'] <= 3)
zscore_change_percent_condition = (df_2020_2024['change_percent_scaled_zscore'] >= -3) & (
    df_2020_2024['change_percent_scaled_zscore'] <= 3)

df_2020_2024_zscore = df_2020_2024_zscore[zscore_close_condition &
                                          zscore_change_percent_condition]

# Scatter plot de los precios de cierre y los retornos diarios (ALL vs. Z-score cleaning)
values = [
    ['max', 'dot', 'blue', 'Máximo', 'top right'],
    ['min', 'dot', 'red', 'Mínimo', 'bottom right'],
]
fig1 = plot_simple_scatter(df_2020_2024_zscore, 'date', 'close', 'Precios de cierre diarios de Tesla con valores atípicos vs. sin (Z-score cleaning) (2020-2024)',
                           'Fecha', 'Precio de cierre', log=False, rect=[], values=values, df_bg=df_2020_2024, df_bg_title='Originales')
fig2 = plot_simple_scatter(df_2020_2024_zscore, 'date', 'change_percent', 'Retornos diarios de Tesla con valores atípicos vs. sin (Z-score cleaning) (2020-2024)',
                           'Fecha', 'Retorno diario', log=False, rect=[], values=values, df_bg=df_2020_2024, df_bg_title='Originales')

fig1.show()
fig2.show()

Gracias a lo cual notamos una menor limpieza que con el método del rango intercuartílico, lo cual puede observarse dado que los valores de los mínimos y máximos de los precios de cierre ajustados y los retornos diarios son mayores que en el caso anterior.

La limpieza puede verse comparando los box plots de los precios de cierre ajustados y los retornos diarios con y sin los valores atípicos:


In [20]:
# Boxplot de los precios de cierre (ALL vs. Z-score cleaning)
fig_orig = plot_simple_boxplot(df_2020_2024, 'close', 'Con outliers',
                               'Precio de cierre', show_points=True, with_mean_std=True)
fig_zscore = plot_simple_boxplot(df_2020_2024_zscore, 'close', 'Sin outliers (Z-score cleaning)',
                                 'Precio de cierre', show_points=True, with_mean_std=True)

fig = plot_subplots([fig_orig, fig_zscore],
                    'Distribución de precios de cierre de Tesla con y sin outliers (2020-2024)', height=600)
fig.show()

# Boxplot de los retornos diarios (ALL vs. Z-score cleaning)
fig_orig = plot_simple_boxplot(df_2020_2024, 'change_percent', 'Con outliers',
                               'Retorno diario', show_points=True, with_mean_std=True)
fig_zscore = plot_simple_boxplot(df_2020_2024_zscore, 'change_percent',
                                 'Sin outliers (Z-score cleaning)', 'Retorno diario', show_points=True, with_mean_std=True)

fig = plot_subplots([fig_orig, fig_zscore],
                    'Distribución de retornos diarios de Tesla con y sin outliers (2020-2024)', height=600)
fig.show()

Gracias a lo cual podemos apreciar la limpieza de _outliers_ realizada para este período con el método de Z-Score.


### 5.3. Comparación de métodos


Si queremos comparar la limpieza realizada por cada método en particular para evaluar la _calidad_ de los datos resultantes y la cantidad de valores atípicos eliminados, podemos hacerlo con el gráfico conjunto de box plots:


In [21]:
# Boxplot de los precios de cierre (ALL vs. IQR cleaning vs. Z-score cleaning)
fig1 = plot_simple_boxplot(
    df_2020_2024, 'close', 'Con outliers', 'Precio de cierre', show_points=True, with_mean_std=True)
fig2 = plot_simple_boxplot(df_2020_2024_iqr, 'close',
                           'Sin outliers (IQR cleaning)', 'Precio de cierre', show_points=True, with_mean_std=True)
fig3 = plot_simple_boxplot(df_2020_2024_zscore, 'close',
                           'Sin outliers (Z-score cleaning)', 'Precio de cierre', show_points=True, with_mean_std=True)

fig = plot_subplots([fig1, fig2, fig3],
                    'Distribución de precios de cierre de Tesla con y sin outliers (2020-2024)', height=700)
fig.show()

# Boxplot de los retornos diarios (ALL vs. IQR cleaning vs. Z-score cleaning)
fig1 = plot_simple_boxplot(df_2020_2024, 'change_percent',
                           'Con outliers', 'Retorno diario', show_points=True, with_mean_std=True)
fig2 = plot_simple_boxplot(df_2020_2024_iqr, 'change_percent',
                           'Sin outliers (IQR cleaning)', 'Retorno diario', show_points=True, with_mean_std=True)
fig3 = plot_simple_boxplot(df_2020_2024_zscore, 'change_percent',
                           'Sin outliers (Z-score cleaning)', 'Retorno diario', show_points=True, with_mean_std=True)

fig = plot_subplots(
    [fig1, fig2, fig3], 'Distribución de retornos diarios de Tesla con y sin outliers (2020-2024)', height=700)
fig.show()

Gracias a lo cual podemos notar, como bien se mencionó anteriormente, que el método del rango intercuartílico es más agresivo en la limpieza de valores atípicos que el método de Z-Score, lo cual se puede ver en la cantidad de _outliers_ eliminados en cada caso.

Por ello mismo, entonces, en los siguientes pasos vamos a trabajar con los datos limpiados con el método del rango intercuartílico.


## 6. Manejo de valores faltantes


En este punto, si bien la idea es realizar el manejo de valores faltantes notando que las columnas `change_percent` y `avg_vol_20` tienen valores faltantes, y posteriormente eliminando esas filas (las 19 primeras), dado que el período seleccionado para realizar el análisis exploratorio y la curación de datos es el comprendido entre el $1$ de _Enero del_ $2020$ y el $15$ de _Abril del_ $2024$, entonces no habrá valores faltantes en el dataset _per se_.

Por este mismo motivo, como anteriormente hicimos la limpieza de valores atípicos y elegimos el método IQR para realizarlo, entonces en este punto vamos a hacer un resample con interpolación lineal para "rellenar" los días faltantes anteriormente eliminados.

Para ello, entonces, primero agreguemos los días faltantes al dataset:


In [22]:
# Agregamos todos los días en el rango 01/01/2020 - 15/04/2024
df_2020_2024_iqr_all = df_2020_2024_iqr.copy()
df_2020_2024_iqr_all = pd.merge(pd.DataFrame(pd.date_range(
    start='2020-01-01', end='2024-04-15', freq='D'), columns=['date']), df_2020_2024_iqr_all, how='left', on='date')

display(pd.DataFrame({'Cantidad de días': [df_2020_2024_iqr.shape[0], df_2020_2024_iqr_all.shape[0]]}, index=[
        'Antes (IQR Cleaning)', 'Después (completo)']).T)

,Antes (IQR Cleaning),Después (completo)
Cantidad de días,1020,1567


Luego, entonces, podemos ver la cantidad de valores faltantes en el dataset nuevo a considerar:


In [23]:
# Bar plot de valores faltantes
# Forma clásica: Missingno
# msno.bar(df_2020_2024_iqr_all, color='tomato', sort='ascending')

# Forma con Plotly
fig = go.Figure()
fig.add_trace(go.Bar(x=df_2020_2024_iqr_all.columns,
              y=df_2020_2024_iqr_all.notnull().sum(), marker_color='tomato'))
fig.update_layout(title='Valores faltantes en el dataset de Tesla',
                  xaxis_title='Columnas', yaxis_title='Cantidad no nula')
fig.show()

Por lo que vamos a rellenarlos realizando un resample con interpolación lineal:


In [24]:
# Interpolamos los valores faltantes
for feature in df_2020_2024_iqr_all.columns[1:]:
    df_2020_2024_iqr_all[feature] = df_2020_2024_iqr_all[feature].interpolate()

# Eliminamos el primer día si llega a ser NaN
df_2020_2024_iqr_all = df_2020_2024_iqr_all.dropna()

# Bar plot de valores faltantes
fig = go.Figure()
fig.add_trace(go.Bar(x=df_2020_2024_iqr_all.columns,
              y=df_2020_2024_iqr_all.notnull().sum(), marker_color='mediumseagreen'))
fig.update_layout(title='Valores faltantes en el dataset de Tesla',
                  xaxis_title='Columnas', yaxis_title='Cantidad no nula')
fig.show()

Y, finalmente, podemos ver cómo se ven los precios de cierre ajustados y los retornos diarios con los valores faltantes interpolados (para comparar con los datos originales del período 2020 - 2024):


In [25]:
# Scatter plot de los precios de cierre y los retornos diarios (ALL vs. IQR cleaning + Interpolación)
values = [
    ['max', 'dot', 'blue', 'Máximo', 'top right'],
    ['min', 'dot', 'red', 'Mínimo', 'bottom right'],
]

fig1 = plot_simple_scatter(df_2020_2024_iqr_all, 'date', 'close', 'Precios de cierre diarios de Tesla originales vs. (IQR cleaning + Interpolación) (2020-2024)',
                           'Fecha', 'Precio de cierre', log=False, rect=[], values=values, df_bg=df_2020_2024, df_bg_title='Originales')
fig2 = plot_simple_scatter(df_2020_2024_iqr_all, 'date', 'change_percent', 'Retornos diarios de Tesla originales vs. (IQR cleaning + Interpolación) (2020-2024)',
                           'Fecha', 'Retorno diario', log=False, rect=[], values=values, df_bg=df_2020_2024, df_bg_title='Originales')

fig1.show()
fig2.show()

Gracias a lo cual podemos notar que la diferencia que se encuentra sigue siendo la misma obtenida anteriormente con la limpieza de valores atípicos con el método IQR. Sin embargo, donde esperamos obtener algunas diferencias más notorias es en los gráficos de box plot, dado que al agregar nuevos datos, entonces las medidas de dispersión y posición de los datos deberían cambiar.


In [26]:
# Boxplot de los precios de cierre (ALL vs. IQR cleaning vs. IQR cleaning + Interpolación)
fig1 = plot_simple_boxplot(df_2020_2024, 'close', 'Original',
                           'Precio de cierre', show_points=True, with_mean_std=True)
fig2 = plot_simple_boxplot(df_2020_2024_iqr, 'close', 'IQR cleaning',
                           'Precio de cierre', show_points=True, with_mean_std=True)
fig3 = plot_simple_boxplot(df_2020_2024_iqr_all, 'close', 'IQR cleaning + Interpolación',
                           'Precio de cierre', show_points=True, with_mean_std=True)

fig = plot_subplots(
    [fig1, fig2, fig3], 'Distribución de precios de cierre de Tesla (ALL vs. IQR cleaning vs. IQR cleaning + Interpolación) (2020-2024)', height=700)
fig.show()

# Boxplot de los retornos diarios (ALL vs. IQR cleaning vs. IQR cleaning + Interpolación)
fig1 = plot_simple_boxplot(df_2020_2024, 'change_percent', 'Original',
                           'Retorno diario', show_points=True, with_mean_std=True)
fig2 = plot_simple_boxplot(df_2020_2024_iqr, 'change_percent', 'IQR cleaning',
                           'Retorno diario', show_points=True, with_mean_std=True)
fig3 = plot_simple_boxplot(df_2020_2024_iqr_all, 'change_percent', 'IQR cleaning + Interpolación',
                           'Retorno diario', show_points=True, with_mean_std=True)

fig = plot_subplots(
    [fig1, fig2, fig3], 'Distribución de retornos diarios de Tesla (ALL vs. IQR cleaning vs. IQR cleaning + Interpolación) (2020-2024)', height=700)
fig.show()

De ese modo, entonces, podemos notar cómo la interpolación genera que los retornos diarios tengan una menor dispersión de los datos alrededor de la mediana y la media, y que los precios de cierre tengan una mayor dispersión a izquierda (valores menores). Esto es claro dado que es en el período $2020$ - $2021$ donde valores más "extremos" se eliminaron por la limpieza aplicada a los retornos.

Por ello mismo, entonces, para los posteriores análisis vamos a trabajar con los datos interpolados.


## 7. Análisis de correlación


Para este punto, lo que nos interesa ahora es analizar la correlación de nuestros datos y evaluar si existe alguna relación lineal entre ellos. La importancia de realizar este análisis radica en que, si existe una correlación alta entre dos variables, entonces una de ellas podría ser eliminada del modelo, dado que no aportaría información adicional a la posterior predicción que quiera realizarse.

Para ello, vamos a analizar la correlación entre todas las variables del dataset (excepto `date`, claro) y ver cómo se relacionan entre sí.


In [27]:
heatmap_cols = ['open', 'high', 'low', 'close',
                'volume', 'change_percent', 'avg_vol_20d']
corr_matrix = df_2020_2024_iqr_all[heatmap_cols].corr()

fig = go.Figure(data=go.Heatmap(z=corr_matrix, x=heatmap_cols,
                y=heatmap_cols, zmin=-1, zmax=1, zhoverformat='.2f', autocolorscale=True))
fig.update_layout(
    title='Correlación entre las variables del dataset de Tesla (2020-2024)', height=600, width=800)
fig.show()

Gracias a lo cual, podemos observar y notar que:

- `open`, `close`, `low` y `high` tienen una correlación perfecta positiva ($1$), lo cual es lógico dado que son los valores de los precios diarios de las acciones de Tesla.
- `avg_vol_20` y `volume` con `open`, `close`, `low` y `high` tienen una correlación fuerte negativa ($-0.76$ y $-0.66$, respectivamente).
- `avg_vol_20` y `volume` tienen una correlación fuerte positiva ($0.81$), lo cual es lógico dado que ambas representan el volumen de acciones negociadas, solo que uno es diario y el otro es el promedio de los últimos $20$ días.
- El resto de los pares de variables no están correlacionadas entre sí.


## 8. Análisis de AutoCorrelación y AutoCorrelación Parcial


En este punto, nos interesa analizar la autocorrelación y la autocorrelación parcial de los precios de cierre ajustados y los retornos diarios de las acciones de Tesla. La importancia de realizar este análisis radica en que, si existe una correlación alta entre una variable y sus valores pasados, entonces se podría considerar la inclusión de _lags_ en el modelo para mejorar la predicción, dado que indican un patrón en los datos.

Para ello mismo, entonces, vamos a realizar los gráficos ACF y PACF para medir y analizar la autocorrelación de las variables `close` y `change_percent`. Miraremos primero los **precios de cierre ajustados**:


In [28]:
# Autocorrelación y Autocorrelación Parcial de los precios de cierre
fig_acf, fig_pacf = plot_acf_pacf(df=df_2020_2024_iqr_all, x='close', nlags=40)
fig_acf.show()
fig_pacf.show()

Gracias a ello, podemos notar que:

- _ACF_: Todas las correlaciones con los primeros $40$ retardos se encuentran por encima del intervalo de confianza (región sombreada), lo que nos indica que los valores actuales de los precios de cierre de Tesla se encuentran fuertemente correlacionados con sus valores pasados (i.e., hay una dependencia temporal en los datos). Además, como la correlación decrece lentamente, entonces podemos decir que existe una tendencia en la serie.
- _PACF_: Podemos notar que las únicas correlaciones fuertes que se encuentran son con el primer retardo (casi $1$) y el segundo ($-0.1$), lo cual nos indica que los valores actuales de los precios de cierre de Tesla están fuertemente correlacionados con sus valores inmediatamente anteriores. Es decir, el precio de cierre de hoy depende fuertemente del precio de cierre de ayer (lo cual, ciertamente, es esperable), y en menor medida del precio de cierre de anteayer.

Este patrón es común en procesos $AR(2)$ (AutoRegresivo de orden $2$), donde cada valor de la serie temporal se explica principalmente como una combinación lineal de los dos valores anteriores más un término de error.


Ahora, de forma totalmente análoga, vamos a analizar los **retornos diarios**:


In [29]:
# Autocorrelación y Autocorrelación Parcial de los retornos diarios
fig_acf, fig_pacf = plot_acf_pacf(
    df=df_2020_2024_iqr_all, x='change_percent', nlags=40)
fig_acf.show()
fig_pacf.show()

Gracias a lo cual, podemos notar que:

- _ACF_: Fuera de la zona sombreada, es decir, superior al intervalo de confianza, se encuentran los retardos $1$ y $2$. Por ello mismo, no se puede decir que exista una tendencia en la serie, pero sí que existe una dependencia temporal inmediata en los datos.
- _PACF_: Podemos notar que la única correlación fuerte que se encuentra es con el primer retardo, lo cual nos indica que la dependencia significativa se explica principalmente por el primer retardo.

Por ello mismo, entonces, podemos decir que los retornos diarios de las acciones de Tesla se pueden modelar como un proceso $AR(1)$, donde no existe una tendencia pero sí una dependencia temporal inmediata en los datos, lo que nos permite explicar los valores actuales de los retornos diarios con sus valores inmediatamente anteriores más un término de error.


## 9. Suavizamiento de Series Temporales


Luego de haber analizado las posibles tendencias y dependencias temporales en los datos, nos vamos a enfocar en el _suavizamiento_ de la serie temporal de los precios de cierre ajustados de las acciones de Tesla para reducir la variabilidad y resaltar las tendencias subyacentes. Para ello, vamos a aplicar dos técnicas distintas:

- Suavizamiento exponencial
- Suavizamiento mediante medias móviles

Veamos cada una de ellas.


### 9.1. Suavizamiento exponencial


En este caso, vamos a aplicar el suavizamiento exponencial, el cual es más sensible a las variaciones recientes de los datos, por lo que es útil para detectar cambios en la tendencia de la serie temporal.

Para ello, vamos a usar el método `SimpleExpSmoothing` de la librería `statsmodels` y vamos a aplicarlo a los precios de cierre ajustados de las acciones de Tesla. Vamos a considerar un _smoothing level_ de $0.2$ para el suavizamiento exponencial.


In [30]:
# Suavizado exponencial simple
close_series_exp = sm.tsa.SimpleExpSmoothing(
    df_2020_2024_iqr_all['close'], initialization_method='estimated').fit(smoothing_level=0.2, optimized=False)

# Gráfico
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_2020_2024_iqr_all['date'], y=df_2020_2024_iqr_all['close'],
                         mode='lines', name='Original', line=dict(color='black', width=1)))
fig.add_trace(go.Scatter(x=df_2020_2024_iqr_all['date'], y=close_series_exp.fittedvalues,
                         mode='lines', name='Suavizado exponencial simple (alpha = 0.2)', line=dict(color='red', width=1)))
fig.update_layout(title='Suavizado exponencial simple de los precios de cierre de Tesla (2020-2024)',
                  xaxis_title='Fecha', yaxis_title='Precio de cierre')
fig.show()

c:\Users\emanu\anaconda3\envs\data_science\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

An unsupported index was provided and will be ignored when e.g. forecasting.



Como podemos observar, el suavizamiento exponencial nos permite ver cómo se comportan los precios de cierre ajustados de las acciones de Tesla en el período $2020$ - $2024$ de una manera más suave y menos volátil, pero manteniendo la tendencia general de los datos y resaltando los cambios en la tendencia de la serie temporal en un período corto de tiempo (es sensible a las variaciones recientes de los datos).

Nos sirve para sacar el ruido de los datos y detectar cambios rápidos en la tendencia de la serie temporal.


### 9.2. Suavizamiento mediante medias móviles


En este caso, vamos a aplicar el suavizamiento mediante medias móviles, el cual es más sensible a las variaciones a largo plazo de los datos, por lo que es útil para detectar cambios en la tendencia de la serie temporal a lo largo del tiempo. Vamos a considerar una ventana de $20$ días para el cálculo de las medias móviles.


In [31]:
# Suavizado con media móvil
close_series_rolling = df_2020_2024_iqr_all['close'].rolling(window=20).mean()

# Gráfico
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_2020_2024_iqr_all['date'], y=df_2020_2024_iqr_all['close'],
                         mode='lines', name='Original', line=dict(color='black', width=1)))
fig.add_trace(go.Scatter(x=df_2020_2024_iqr_all['date'], y=close_series_rolling,
                         mode='lines', name='Media móvil (ventana = 20)', line=dict(color='red', width=1)))
fig.update_layout(title='Suavizado con media móvil de los precios de cierre de Tesla (2020-2024)',
                  xaxis_title='Fecha', yaxis_title='Precio de cierre')
fig.show()

Con ello, podemos notar cómo se comportan los precios de cierre ajustados de las acciones de Tesla en el período $2020$ - $2024$ de una manera más suave y menos volátil, pero manteniendo la tendencia general de los datos y resaltando los cambios en la tendencia de la serie temporal a lo largo del tiempo (notar que es menos sensible a las variaciones recientes de los datos dado que se calcula con una ventana de $20$ días, por lo que los cambios presentan un "delay" en la visualización).

Nos sirve para sacar el ruido de los datos y detectar cambios lentos en la tendencia de la serie temporal.


### 9.3. Comparación de métodos de suavizamiento


Finalmente, y a fines comparativos, vamos a mostrar cómo se ven los precios de cierre ajustados de las acciones de Tesla en el período $2020$ - $2024$ con ambos métodos de suavizamiento aplicados:


In [32]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_2020_2024_iqr_all['date'], y=df_2020_2024_iqr_all['close'],
                         mode='lines', name='Original', line=dict(color='black', width=1)))
fig.add_trace(go.Scatter(x=df_2020_2024_iqr_all['date'], y=close_series_exp.fittedvalues,
                         mode='lines', name='Suavizado exponencial simple (alpha = 0.2)', line=dict(color='red', width=1)))
fig.add_trace(go.Scatter(x=df_2020_2024_iqr_all['date'], y=close_series_rolling,
                         mode='lines', name='Media móvil (ventana = 20)', line=dict(color='blue', width=1)))
fig.update_layout(title='Suavizado exponencial simple vs. Media móvil de los precios de cierre de Tesla (2020-2024)',
                  xaxis_title='Fecha', yaxis_title='Precio de cierre')
fig.show()

Es así como podemos observar las diferencias entre ambos métodos de suavizamiento y cómo resalta cada uno de ellos las tendencias de la serie temporal de los precios de cierre, siendo el suavizamiento exponencial el idóneo para notar cambios recientes en los precios y el suavizamiento mediante medias móviles el idóneo para notar cambios a mediano y largo plazo en los precios.


## 10. Análisis de estacionalidad


En este último punto, el objetivo es descomponer nuestra serie temporal de los precios de cierre en sus componentes de _tendencia_, _estacionalidad_ y _ruido_ para poder analizar cada uno de ellos por separado y entender cómo se comportan nuestros datos.

Para ello, vamos a usar el método `seasonal_decompose` de la librería `statsmodels` y vamos a aplicarlo a los precios de cierre ajustados de las acciones de Tesla en el período $2020$ - $2024$. La descomposición se realizará con el modelo aditivo y un período de estacionalidad de un cuatrimestre (aproximadamente $90$ días).


In [33]:
# Descomposición de los precios de cierre
asd = df_2020_2024_iqr_all.copy().set_index('date')
decomposition = sm.tsa.seasonal_decompose(
    asd['close'], model='additive', period=90)

# Gráficos
fig = make_subplots(rows=4, cols=1, subplot_titles=[
                    'Original', 'Tendencia', 'Estacionalidad', 'Ruido'], shared_xaxes=True)

fig.add_trace(go.Scatter(x=asd.index, y=decomposition.observed,
                         mode='lines', name='Original', line=dict(color='black')), row=1, col=1)
fig.add_trace(go.Scatter(x=asd.index, y=decomposition.trend,
                         mode='lines', name='Tendencia', line=dict(color='blue')), row=2, col=1)
fig.add_trace(go.Scatter(x=asd.index, y=decomposition.seasonal,
                         mode='lines', name='Estacionalidad', line=dict(color='green')), row=3, col=1)
fig.add_trace(go.Scatter(x=asd.index, y=decomposition.resid,
                         mode='lines', name='Ruido', line=dict(color='red')), row=4, col=1)

fig.update_layout(title='Descomposición de los precios de cierre de Tesla (2020-2024)',
                  xaxis_title='Fecha', height=1000)
fig.show()

Gracias a lo cual podemos analizar cada componente por separado y concentrarnos en ver la tendencia y el ruido para determinados períodos de estacionalidad (en este caso, $90$ días fueron los considerados). Con ello, podemos notar que la tendencia actual es bajista, que la estacionalidad es mayormente positiva y que el ruido es irregular a lo largo del tiempo y no sigue un patrón claro (hay momentos de mayor y menor volatilidad).
